In [1]:
import sys
sys.path.append('../..')
import cortex_etl as c_etl

In [ ]:
ma_v1 = c_etl.analysis_initial_processing('/gpfs/bbp.cscs.ch/project/proj83/home/isbister/physiology_2023/cortex_etl/configs/3-ThalamicStimuli/3-ThalamicStimuli-MegaScan2Scalings.yaml', loglevel="ERROR")
a_hex0_v1 = ma_v1.hex0_spikes

ma_v2 = c_etl.analysis_initial_processing('/gpfs/bbp.cscs.ch/project/proj83/home/isbister/physiology_2023/cortex_etl/configs/post_physiology_2023/campaigns/schneider_mizell/1_whisker_deflections/1_3_whisker_deflections_scan_16_4_24.yaml', loglevel="ERROR")
a_hex0_v2 = ma_v2.hex0_spikes

/gpfs/bbp.cscs.ch/home/isbister/cortex_etl_jupyter_04_01_23/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
c_etl.evoked_processing(a_hex0_v1)
c_etl.evoked_processing(a_hex0_v2)

In [ ]:
import pandas as pd
import numpy as np

merged_s_ratios = pd.merge(a_hex0_v1.custom['s_ratios'], a_hex0_v2.custom['s_ratios'], on=['ca', 'desired_connected_proportion_of_invivo_frs', 'depol_stdev_mean_ratio', 'neuron_class'], suffixes=('_v1', '_v2')) 
merged_s_ratios = merged_s_ratios[np.logical_and(merged_s_ratios.ratio_v1 != np.inf, merged_s_ratios.ratio_v2 != np.inf)]
merged_s_ratios["ratio_diff"] = merged_s_ratios['ratio_v2'] - merged_s_ratios['ratio_v1']
merged_s_ratios["ratio_diff_proportion"] = merged_s_ratios['ratio_v2'] / merged_s_ratios['ratio_v1']
merged_s_ratios['c'] = merged_s_ratios.apply(lambda row: c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES[row['neuron_class']]["color"], axis=1)
merged_s_ratios['marker'] = merged_s_ratios.apply(lambda row: c_etl.LAYER_EI_NEURON_CLASS_MARKERS[row['neuron_class']], axis=1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.figure()
upper_lim = np.nanmax([merged_s_ratios['ratio_v1'].max(skipna=True), merged_s_ratios['ratio_v2'].max(skipna=True)])
lims = [0, upper_lim]

for _, row in merged_s_ratios.iterrows():
    plt.scatter(row['ratio_v1'], row['ratio_v2'], s=1.5, c=row['c'], marker=row['marker'])
plt.plot(lims, lims)
ax = plt.gca()
# ax.set_xlim(lims); ax.set_ylim(lims)
ax.set_xscale('log'); ax.set_yscale('log')
ax.set_aspect('equal', 'box')
plt.show()
plt.close()

In [ ]:
import seaborn as sns
def horizontal_box_plot(data, y, x, figsize, order, palette, labels, line_loc, figdir, lims=[], hue=None, width=.5, gap=0, dodge=False):
    plt.figure(figsize=figsize)
    # sns.violinplot(x="neuron_class", y="true_mean_conductance_change", data=merged_by_nc, palette=nc_c, order=nc_order)
    # sns.stripplot(y="neuron_class", x="true_mean_conductance_change", data=merged_by_nc, jitter=True, s=1.2, palette=nc_c, order=nc_order, orient='h')
    sns.boxplot(y=y, x=x, hue=hue, data=data, palette=palette, order=order, orient='h', width=width, linewidth=.2, showfliers=False, gap=gap, dodge=dodge)
    ax = plt.gca()
    ax.set_yticklabels(labels)
    ax.set_ylabel('')
    ax.set_xlabel(stat_labels[x] + " " + stat_units[x])
    ax.axvline(x=line_loc, color='black', linestyle='dashed', linewidth=.3, zorder=-4)
    
    if len(lims):
        ax.set_xlim(lims)
    
    plt.tight_layout()
    plt.savefig(figdir + x + '.pdf', bbox_inches='tight')
    plt.show()
    plt.close()
    
figsize=(3,2)
figdir = ""
# nc_order = ['L1_INH', 'L23_INH', 'L4_INH', 'L5_INH', 'L6_INH', 'L23_EXC', 'L4_EXC', 'L5_EXC', 'L6_EXC']
# nc_c = ['blue', 'blue', 'blue', 'blue', 'blue', 'red', 'red', 'red', 'red']
# layer_labels = ['L1', 'L23', 'L4', 'L5', 'L6', 'L23', 'L4', 'L5', 'L6']


nc_order = ['L23_INH', 'L4_INH', 'L5_INH', 'L23_EXC', 'L4_EXC', 'L5_EXC']
nc_c = ['blue', 'blue', 'blue', 'red', 'red', 'red']
layer_labels = ['L23', 'L4', 'L5', 'L23', 'L4', 'L5']
stat_labels = {
    "ratio_diff": "$\Delta_{v2 - v1}$ evoked ratio",
    "ratio_diff_proportion": "$v2/v1$ evoked ratio"
}
stat_units = {
    "ratio_diff": "",
    "ratio_diff_proportion": ""
}

# print(merged_s_ratios['neuron_class'])
horizontal_box_plot(merged_s_ratios, "neuron_class", "ratio_diff", figsize, nc_order, nc_c, layer_labels, 0, figdir, hue=None, width=.8, gap=0)
horizontal_box_plot(merged_s_ratios, "neuron_class", "ratio_diff_proportion", figsize, nc_order, nc_c, layer_labels, 1, figdir)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import numpy as np
def evoked_ratios_line_plot(a, comp_ncs, stat_key='ratio'):
    
    s_ratios = a.custom['s_ratios']
    mean_ratio_over_ncs_by_sim = a.custom['mean_ratio_over_ncs_by_sim']

    plt.figure(figsize=(8.2*0.3, 8.2*0.3*0.5))
    ax = plt.gca()
    ax.set_facecolor('lightgrey')
    cmap = sns.color_palette("seismic", as_cmap=True)

    max_abs_diff = np.max(np.abs(mean_ratio_over_ncs_by_sim['mean_ratio_over_ncs_by_sim_diff_to_vivo']))
    c_range_width = max_abs_diff * 2

    for i, simulation_id in enumerate(s_ratios.simulation_id.unique()):

        sim_nc_s_ratios = s_ratios.etl.q(simulation_id=simulation_id, neuron_class=comp_ncs)
        sim_mean_ratio_over_ncs = mean_ratio_over_ncs_by_sim.etl.q(simulation_id=simulation_id)['mean_ratio_over_ncs_by_sim_diff_to_vivo']
        
        if stat_key == 'ratio':
            c_val = (sim_mean_ratio_over_ncs / c_range_width) + 0.5
            c=cmap(c_val)
        elif stat_key == 'ratio_normalized':
            c='k'

        plt.plot(range(len(comp_ncs)), sim_nc_s_ratios[stat_key], c=c, lw=.2)


    # if (normalisation_type == "none"):
    #     plt.plot(range(len(neuron_classes)), vivo_nc_ratios, c='w', lw=1, marker='o', ms=3)
    #     plt.plot(range(len(neuron_classes)), silico_mean_ratios_across_sims.etl.q(neuron_class=neuron_classes), c='k', lw=1, marker='o', ms=3)
    #     ax.set_ylim([0.0, 200.0])

    # elif (normalisation_type == "mean_normalise"):
    #     plt.plot(range(len(neuron_classes)), mean_normalise(vivo_nc_ratios), c='w', lw=1, marker='o', ms=3)
    #     plt.plot(range(len(neuron_classes)), mean_normalise(silico_mean_ratios_across_sims.etl.q(neuron_class=neuron_classes)), c='k', lw=1, marker='o', ms=3)
    #     ax.set_ylim([0.0, 5.0])



    ax.yaxis.set_major_locator(ticker.MultipleLocator(200))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(50))
    ax.set_xticks(range(len(comp_ncs)), labels=[c_etl.neuron_class_label_map[nc] for nc in comp_ncs])
    ax.set_ylabel('Evok/spont\nratio', labelpad=-10.0)

    plt.tight_layout()
    plt.show()
    plt.savefig(str(a.figpaths.evoked) + "/" + stat_key + "_EvokedRatios.pdf")
    plt.close()


def evoked_ratios_plots(a, comp_ncs):
    silico_mean_ratios_across_sims = None
    evoked_ratios_line_plot(a, comp_ncs)
    # c_etl.evoked_ratios_line_plot(a, comp_ncs, normalisation_type='mean_normalise')
    

"""
Entry function
"""
def evoked_analysis(a):

    comp_ncs, \
    vivo_df, \
    rp_psth_df, \
    svo_psth_df = evoked_processing(a)
    
    print(vivo_df)
    
    print(rp_psth_df)

#     evoked_plots(a, comp_ncs, vivo_df, rp_psth_df, svo_psth_df)


"""
Processing
"""
def evoked_processing(a):

    c_etl.create_dirs(a)

    vivo_df, rp_psth_df, svo_psth_df = c_etl.preprocess_vivo_dfs()
    comp_ncs = c_etl.select_comparison_neuron_classes(a)

    c_etl.compare_latencies_and_decay(a, vivo_df)
    c_etl.create_sim_mask(a, ["ALL"] + comp_ncs)
    c_etl.compare_evoked_ratios(a, comp_ncs)
    c_etl.merge_comparison_and_mask(a)
    c_etl.filter_dfs_by_window(a)

    return comp_ncs, vivo_df, rp_psth_df, svo_psth_df

"""
Plots
"""
def evoked_plots(a, comp_ncs, vivo_df, rp_psth_df, svo_psth_df):
    evoked_ratios_plots(a, comp_ncs)
    # evoked_heatmaps(a)
    # compare_time_courses(a, vivo_df, silico_bin_size=0.5, silico_sigma=2)
    # psth_plots(a, comp_ncs, rp_psth_df, svo_psth_df)
    
    
evoked_analysis(a)

In [ ]:
# !grep -r compare_metrics_to_in_vivo_for_neuron_classes ../../cortex_etl/ 